# Preface
> The questions/exercises given for the assignment has been repeated in this notebook, they have been put in bold. To easily distinguish our answers and explanations have been indented, or printed from the executed code. 

## Imports
> The following imports are needed throughout the notebook:

In [2]:
import csv
import os
import nltk
import networkx as nx
import matplotlib.pyplot as plt
import re
import string
import statistics
import numpy as np
import random
import math
import pandas as pd
import json
import urllib.request

from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from wordcloud import WordCloud
from community import community_louvain
from collections import Counter
from os import listdir
from bs4 import BeautifulSoup

## Data

> The downloaded files can be found in the folders `dc` and `marvel`, in our [github project](https://github.com/pete414n/02805SG/tree/main/assignment2).

# Basic Stats

**Write a short paragraph describing the network. The paragraph should contain the following information**
- **The number of nodes and links.**
- **The average, median, mode, minimum and maximum value of the network's in-degree.s And of the out-degrees.**

> This is just a placeholder for the short paragraph

**We also want the degree distributions and a plot of the network**
- **Create in- and out-going degree distributions as described in Lecture 5.**

- **Estimate the slope of the incoming degree distribtion as described in Lecture 5.**

- **Plot the network using the Force Atlas algorithm as described in Lecture 5.**

# Communities

**Identify the communities in one or both of the superhero universes (DC/Marvel) as described in Week 7.**

**Visualize the communities by coloring the graph's nodes according to community affiliation - also as described in Week 7.**

**Write a paragraph about your work to identify the communities. The paragraph should include**
- **Information about the algorithm you used to find communities.**
- **The value of modularity for your network.**
- **The number of communities and their sizes (in terms of number of nodes).**

> This is just a placeholder for the paragraph

**For the 10 largest communities, process the text in order to create a TF-IDF vector for each community. Explain in your own words how TF-IDF works.**

> This is just a placeholder for the paragraph explaining how TF-IDF works

**Then visualize the TF-IDF vectors using wordclouds (as described in Week 7). Remember to comment on your word-clouds (e.g. in the figure captions): Do the wordclouds/TF-IDF lists enable you to understand the communities you have found (or is it just gibberish)? Justify your answer.**

> This is just a placeholder for the paragraph for our answer and the justification

# Sentiment

**First calculate the sentiment for all character pages. Calculate the mean value of the sentiment and the 10th and 90th percentile values for the sentiment. Then create a nice histogram displaying the distribution of sentiment values for all pages. Indicate the mean, etc on that histogram.**

> We start by preparing the data from the files that have been given for the exercise:

In [4]:
# Create an empty list to store tuples of words and their 
# average happiness score
sent_list = []

# Read in the .tsv file
with open("dataset_s1.tsv", "r", encoding="utf8") as sent_file:
    tsv_reader = csv.DictReader(sent_file, delimiter="\t")
    # For each sentiment in the file, save the word and average happiness in a tuple
    # and add it to the list
    for sent in tsv_reader:
        word = sent["word"]
        average = sent["happiness_average"]
        sent_list.append((word, average))

In [ ]:
# Function for calculating the sentiment of a file from the frequency distribution for that file
def sentiment(tokens):
    # Total sentiment score of file
    sent_sum = 0.0
    # Total number of occurences of words
    occ_sum = 0
    
    # For each token and associated number of occurences
    for token, occ in tokens.items():
        # If the token is in the given list of words with rated happiness
        if token in [i[0] for i in sent_list]:
            # For the token and score in the list of words with rated happiness
            for (token, score) in sent_list:
                # Multiply the average happiness score with the number of occurences
                sent_sum += (float(score) * occ)
                # Add the number of occurences to the total number of occurences
                occ_sum += occ
    return sent_sum / occ_sum

**Now grab the good/bad character names (from this file, see Week 8 for details) and calculate the same stats for only good/bad characters; also plot the histograms for the good/bad group and indicate the values of the mean, etc.**

**Finally use the label permutation test (see Week 5) to investigate the two hypotheses below**
- **H1: Good characters have a higher averge value for sentiment than a similarly sized set of randomly selected characters.**
- **H2: Bad characters have a lower average value for sentiment than a similarly sized set of randomly selected characters.**

> This is just a placeholder for the paragraph for our findings in regards to H1 and H2

**Write a short paragraph reflecting on your findings.**

> This is just a placeholder for the paragraph